## Setup

In [1]:
using PyCall
using SoleAudio
using MFCC

af = pyimport("audioflux")
librosa = pyimport("librosa")
scipy= pyimport("scipy")
np = pyimport("numpy")

sr_src = 16000
x, sr = librosa.load("/home/riccardopasini/Documents/Aclai/Datasets/SpcDS/SpcDS_gender_1000_60_100/WavFiles/common_voice_en_23616312.wav", sr=sr_src, mono=true)
FFTLength = 256
mel_num = 26

# convert to Float64
x = Float64.(x)

96001-element Vector{Float64}:
 -0.035614013671875
  0.00787353515625
  0.045684814453125
  0.064483642578125
  0.051544189453125
  0.032012939453125
  0.0406494140625
  0.068817138671875
  0.07696533203125
  0.041259765625
  ⋮
  0.0050048828125
  0.005462646484375
  0.0059814453125
  0.00592041015625
  0.00543212890625
  0.005126953125
  0.004852294921875
  0.00469970703125
  0.000518798828125

# Stft

### SoleAudio

In [3]:
setup = signal_setup(
    sr=sr,
    # fft
    window_type=:hann,
    window_length=FFTLength,
    overlap_length=Int(round(FFTLength * 0.500)),
    window_norm=:false,
    # spectrum
    freq_range=Int[0, sr / 2],
    spectrum_type=:power,
)

data = signal_data(
    x=x
)

takeFFT(data, setup)

data.stft.stft'

749×129 adjoint(::Matrix{Float64}) with eltype Float64:
 0.0125472    0.00218127  0.412762   1.3521     …  3.44674e-9   3.84944e-9
 0.00137271   0.013376    0.235023   0.64863       2.35531e-9   4.39916e-11
 0.0115559    0.0150591   0.0364172  0.460982      2.38323e-8   3.52941e-8
 2.67995e-6   0.041357    0.430647   0.88339       6.33675e-10  5.38142e-9
 0.00193667   0.0241498   0.620324   1.06114       4.41929e-10  3.82594e-10
 0.0228692    0.00152387  0.369166   0.732187   …  3.88656e-10  9.63737e-12
 0.0297596    0.021313    0.0374478  0.305619      1.96729e-9   1.01384e-9
 0.0142893    0.0147661   0.0385542  0.0379836     4.26155e-9   1.4765e-8
 0.000211095  0.0202506   0.0896516  0.128588      8.90103e-10  4.91196e-9
 3.28872e-5   0.00252675  0.112178   0.18456       9.71086e-9   2.54718e-10
 ⋮                                              ⋱               
 0.00312613   0.00190986  0.233521   1.08932    …  6.25766e-9   3.10301e-9
 0.00308284   9.94183e-5  0.0427588  0.708789      

### AudioFlux

In [12]:
l_bft_obj = af.BFT(
    num=129,
    radix2_exp=Int64(log2(FFTLength)),
    samplate=sr,
    low_fre=0.0,
    high_fre=sr / 2,
    window_type=af.type.WindowType.HANN,
    slide_length=round(Integer, FFTLength * 0.500),
    scale_type=af.type.SpectralFilterBankScaleType.LINEAR,
    style_type=af.type.SpectralFilterBankStyleType.SLANEY,
    data_type=af.type.SpectralDataType.POWER,
    is_reassign=false
)
l_spec_arr = l_bft_obj.bft(x, result_type=1)

l_spec_arr'

749×129 adjoint(::Matrix{Float32}) with eltype Float32:
 0.0123688    0.00240557  0.413322   1.3649     …  3.16014f-9   4.27342f-9
 0.00135975   0.0133003   0.231275   0.653773      2.83098f-9   1.48394f-10
 0.0114607    0.0152416   0.0360091  0.465054      2.34575f-8   3.60848f-8
 5.85557f-6   0.0409189   0.436406   0.884031      6.16921f-10  5.35307f-9
 0.00172068   0.0247849   0.623909   1.07076       4.40205f-10  3.69934f-10
 0.0223057    0.00155754  0.369305   0.738694   …  4.49162f-10  1.33366f-12
 0.0296132    0.0212974   0.0369165  0.304195      2.00174f-9   1.05912f-9
 0.0144823    0.0148994   0.038938   0.0391045     4.37681f-9   1.45576f-8
 0.000241525  0.0204635   0.0907899  0.128806      9.34638f-10  4.94588f-9
 1.5766f-5    0.00273468  0.113115   0.18578       9.76456f-9   2.86674f-10
 ⋮                                              ⋱               
 0.00318086   0.0019212   0.23214    1.09379    …  6.35926f-9   2.97076f-9
 0.00328934   9.27041f-5  0.0417649  0.717787     

### Librosa

In [32]:
ST = np.abs(librosa.stft(
    y=x,
    n_fft=FFTLength,
    hop_length=round(Integer, FFTLength * 0.500),
    window=scipy.signal.windows.hann
))

ST'

751×129 adjoint(::Matrix{Float64}) with eltype Float64:
 0.113458    0.18154     0.290533   0.419855  …  0.0284675    0.0284587
 0.111215    0.0490466   0.642901   1.16829      5.62878e-5   6.53392e-5
 0.0368748   0.115327    0.480911   0.808562     5.30821e-5   1.24103e-5
 0.107054    0.123457    0.189761   0.681949     0.00015329   0.000190077
 0.00241984  0.202284    0.66061    0.940229     2.48935e-5   7.31864e-5
 0.041481    0.157432    0.789879   1.03478   …  2.09963e-5   1.93044e-5
 0.149351    0.0394657   0.607705   0.859473     2.11738e-5   1.2354e-6
 0.172085    0.145936    0.192137   0.551539     4.46965e-5   3.25387e-5
 0.120342    0.122063    0.197327   0.197748     6.61606e-5   0.00012065
 0.0155411   0.143051    0.301314   0.358896     3.05817e-5   7.03429e-5
 ⋮                                            ⋱               
 0.0573527   0.00962833  0.204364   0.847223     9.65961e-5   3.32566e-6
 0.0348884   0.122062    0.728519   1.31735      1.35116e-5   4.61184e-5
 0.010